In [1]:
from sympy import *

In [2]:
x = Symbol('x')
k0 = Symbol('k0')
k1 = Symbol('k1')
k2 = Symbol('k2')

f1 = exp(-(k0+k1) * x)
# f2 = (k1 / (k2 - k1)) * (exp(-k1 * x) - exp(-k2 * x))
f2 = ((exp(-k2* x) - exp(x *(-(k0 + k1)))))#/(k0 + k1 - k2)
# f2
f = f1 * f2
f



(-exp(x*(-k0 - k1)) + exp(-k2*x))*exp(x*(-k0 - k1))

In [3]:
d = f.diff(x)
# d = simplify(d)
d

(-k0 - k1)*(-exp(x*(-k0 - k1)) + exp(-k2*x))*exp(x*(-k0 - k1)) + (-k2*exp(-k2*x) - (-k0 - k1)*exp(x*(-k0 - k1)))*exp(x*(-k0 - k1))

In [5]:
max = solve(d, x)[0]
max

(-log((k0 + k1 + k2)/(k0 + k1)) + log(2))/(k0 + k1 - k2)

In [55]:
maxs = logcombine(max, True)

In [61]:
powsimp(logcombine(f.subs(x, maxs), True), force=True)

(((2*(k0 + k1)/(k0 + k1 + k2))**(-1/(k0 + k1 - k2)))**k2 - ((2*(k0 + k1)/(k0 + k1 + k2))**(1/(k0 + k1 - k2)))**(-k0 - k1))*((2*(k0 + k1)/(k0 + k1 + k2))**(1/(k0 + k1 - k2)))**(-k0 - k1)

In [173]:
limit(max, k1, oo)

0

In [131]:
limit(max, k2, 0)

log(2)/(k0 + k1)

In [174]:
f1 = Function('f1', real=True)
f2 = Function('f2', real=True)
t, C1, C2= symbols("t C1 C2", real=True)
k0 = Symbol('k0', real=True)
k1 = Symbol('k1', real=True)
k2 = Symbol('k2', real=True)
c0 =  Symbol('c0', real=True)

# eq1 = Eq(diff(f1(t), t), -k1*f1(t))

# f1 = c0*exp(-(k0+k1)*t)
eq1 = Eq(diff(f1(t), t), -(k0+k1)*f1(t))
eq2 = Eq(diff(f2(t), t), k1*f1(t) - k2*f2(t))


# sols = dsolve([eq1, eq2])
# # sols[0]
# refine(sols[0], Q.positive(k0))

# constants = solve(sols.subs(t, 0).subs(f1(0), c0), {C1})
# constants

# # solution = dsolve([f1(x).diff(x) + k1*f1(x), f2(x).diff(x) - k1*f1(x) + k2*f2(x)], [f1(x), f2(x)], ics={f1(0):1, f2(0):0})
# solution = dsolve([f1(x).diff(x) + k1*f1(x), f2(x).diff(x) - k1*f1(x) + k2*f2(x)])

In [104]:
constants = solve([sols[0].subs(t, 0).subs(f1(0), 1), sols[1].subs(t, 0).subs(f1(0), 1).subs(f2(0), 0)], {C1, C2})

In [105]:
exp(-t*(k0 + k1))*(exp(-t*(k0 + k1)) + exp(-kd*t)*(k0*t + k1*t - kd*t - 1))

[]

In [43]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from scipy.integrate import odeint




In [44]:
k0 = 4.1e5
kd = 1.86e5
kq = 1.31e10



In [45]:
def lower_lim(conc):
    k1 = kq * conc
    return (np.log(2) - np.log((k0+k1+kd) / (k0+k1)))/ (k0+k1-kd)


In [46]:
def upper_lim(conc):
#     k1 = kq * conc
    x = np.linspace(0, 4e-6, 5000)
    
    maxima = np.zeros(conc.shape[0])
    
    for i, k1 in enumerate(kq*conc):
        df =  np.exp(-x*(k0 + k1))*(np.exp(-x*(k0 + k1)) + np.exp(-kd*x)*(k0*x + k1*x - kd*x - 1))
        m = np.argmax(df)
        
        maxima[i] = x[m]
        
        
    return maxima


In [252]:
def general(conc, kex, k_ex=0, A0=0, t_max=4e-6, intensity=False, curves=False, norm_curves=False, t_points=1000):
    
    def Tall(A):
        result = (1 - np.power(10, -A)) / (A * np.log(10))
        result[np.isnan(result)] = 1
        return result
    
    def solve(conc, t, *args):
        Xan, Ex, DR, DF = conc
        k1_var = args[0]

        dcXan_dt = -(k0 + k1_var) * Xan
        dcEx_dt = + k1_var * Xan  - (kex + kd) * Ex + k_ex * DR
        dcDR_dt = + kex * Ex - kd * DR - k_ex * DR
        # DF simulation
        dcDF_dt = kTT * Xan * DR - kdel * DF

        return [dcXan_dt, dcEx_dt, dcDR_dt, dcDF_dt]
    
    def integrate(k1, times, init_values):
        result = odeint(solve, init_values, times, args=(k1,))
        Xan = result[:, 0]
        Ex = result[:, 1]
        DR = result[:, 2]
        DF = result[:, 3]
        return Xan, Ex, DR, DF
    
    def _max(k1, t):
# #         limit_t = t_max
# #         num_p = 100
#         t = np.linspace(0, t_max, t_num)
        
        Xan, Ex, DR = integrate(k1, t, [1, 0, 0])
        DF_corr = Xan * DR * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if not curves:
            t_max_idx = np.argmax(DF_corr)
            min_idx = t_max_idx - 1 if t_max_idx > 0 else 0
            t_part = np.linspace(t[min_idx], t[t_max_idx + 1], times.shape[0])
            Xan, Ex, DR = integrate(k1, t_part, [Xan[min_idx], Ex[min_idx], DR[min_idx]])
        
        DF_corr = Xan * DR * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if curves:  # return the all curve, 
            return DF_corr / np.max(DF_corr) if norm_curves else DF_corr
        
        return t_part[np.argmax(DF_corr)] if not intensity else np.max(DF_corr)
    
    t_num = t_points if curves else 100
#     t_max = 4e-6
    
    times = np.linspace(0, t_max, t_num)
    
    maxima = np.zeros(conc.shape[0] if not curves else (conc.shape[0], times.shape[0]))
    
        
    for i, k1 in enumerate(kq*conc):
        maxima[i] = _max(k1, times)
        
    return maxima

In [211]:
# slow DF process kinetic model
def general_2(conc, kdel=1, kTT=1, A0=0, t_max=4e-6, intensity=False, curves=False, norm_curves=False):
    
    def Tall(A):
        result = (1 - np.power(10, -A)) / (A * np.log(10))
        result[np.isnan(result)] = 1
        return result
    
    def solve(c, t, *args):
        Xan, Ex, DR = c
        k1_var = args[0]

#         dcXan_dt = -(k0 + k1_var) * Xan
#         dcEx_dt = + k1_var * Xan  - (kex + kd) * Ex
#         dcDR_dt = + kex * Ex - kd * DR

        dcXan_dt = -(k0 + k1_var) * Xan
        dcEx_dt = + kTT * Xan * DR  - kdel * Ex
        dcDR_dt = + k1_var * Xan - kd * DR

        return [dcXan_dt, dcEx_dt, dcDR_dt]
    
    def integrate(k1, times, init_values):
        result = odeint(solve, init_values, times, args=(k1,))
        Xan = result[:, 0]
        Ex = result[:, 1]
        DR = result[:, 2]
        DF = Xan * DR
        return Xan, Ex, DR
    
    def _max(k1, t):
# #         limit_t = t_max
# #         num_p = 100
#         t = np.linspace(0, t_max, t_num)
        
        Xan, Ex, DR = integrate(k1, t, [1, 0, 0])
        DF_corr = Ex * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if not curves:
            t_max_idx = np.argmax(DF_corr)
            min_idx = t_max_idx - 1 if t_max_idx > 0 else 0
            t_part = np.linspace(t[min_idx], t[t_max_idx + 1], times.shape[0])
            Xan, Ex, DR = integrate(k1, t_part, [Xan[min_idx], Ex[min_idx], DR[min_idx]])
        
        DF_corr = Ex * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if curves:  # return the all curve, 
            return DF_corr / np.max(DF_corr) if norm_curves else DF_corr
        
        return t_part[np.argmax(DF_corr)] if not intensity else np.max(DF_corr)
    
    t_num = 1000 if curves else 100
#     t_max = 4e-6
    
    times = np.linspace(0, t_max, t_num)
    
    maxima = np.zeros(conc.shape[0] if not curves else (conc.shape[0], times.shape[0]))
        
    for i, k1 in enumerate(kq*conc):
        maxima[i] = _max(k1, times)
        
    return maxima

In [253]:
conc = np.linspace(1e-10, 7e-4, 100)
ll = lower_lim(conc)

# x = np.linspace(0, 4e-6, 5000)
ul = upper_lim(conc)

var_ = general(conc, 1.5e6, k_ex=0, A0=0)
var2 = general_2(conc, 2.5e6, 1e10, A0=0)
plt.plot(conc, var_, label='true', lw=3)
plt.plot(conc, ul, label='upper', lw=3)
plt.plot(conc, ll, label='lower', lw=3)
for d in np.linspace(3e5, 5e6, 10):
    plt.plot(conc, general_2(conc, d, 1e11, A0=0), label=f'kdelay={d}', color='k')



# plt.plot(conc, ul - ll, label='diff')
plt.legend()
plt.xlabel('x label')
plt.ylabel('y label')


# plt.savefig('figure.png', dpi=1000, format='png', transparent=True)
plt.show()

In [1]:
_c = np.linspace(1e-8, 6e-4, 1000)
# _c = np.logspace(-8, -3, 1000)

t_points = 1000
t_max = 6e-6

matrix = general(_c, 1.5e+06, k_ex=0, A0=0.25, t_max=t_max, curves=True, norm_curves=False, t_points=t_points)
# matrix = general_2(_c, 2.5e6, kTT=1e12, A0=0, t_max=6e-6, curves=True, norm_curves=False)

times = np.linspace(0, t_max, t_points)

matrix /= np.max(matrix)  # normalize to global maximum

curve_x = _c
curve_y = np.zeros(_c.shape[0]) # max time
curve_z = np.zeros(_c.shape[0]) # max intensity

# maxima curve
for i in range(_c.shape[0]):
    curve_y[i] = times[np.argmax(matrix[i])]
    curve_z[i] = np.max(matrix[i])

# matrix /= np.max(matrix)  # normalize to global maximum


x, y = np.meshgrid(_c, times)  # needed for pcolormesh to correctly scale the image

from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mtick

%matplotlib qt

# X, Y, Z = axes3d.get_test_data(0.2)

# Normalize to [0,1]
norm = plt.Normalize(matrix.min(), matrix.max())
colors = cm.inferno_r(norm(matrix.T))
rcount, ccount, _ = colors.shape

fig = plt.figure()
ax = fig.gca(projection='3d')
# surf = ax.plot_surface(x, y, matrix.T, rcount=1, ccount=50,
#                        facecolors=colors, shade=False, linewidth=0.5)
# surf.set_facecolor((0,0,0,0))
wf = ax.plot_wireframe(x, y, matrix.T, rstride=0, cstride=27, linewidth=0.9, color='k')

curve_plot = ax.plot3D(curve_x, curve_y, curve_z, zdir='z', color='green', lw=5, zorder=10)

levels = np.linspace(-0.01, 1, 6)
cm = ax.contourf(x, y, matrix.T, zdir='z', offset=0, levels=levels, cmap='inferno_r')


# cc = ax.contourf(x, y, matrix.T, zdir='y', offset=-0.1)

# ax.colorbar(label='Relative intensity')

# plot data matrix D

# plt.pcolormesh(x, y, matrix.T, cmap='inferno_r')
# plt.colorbar(label='Relative intensity')
# plt.title("Data matrix $D$")
# plt.ylabel('Time [s]')
# plt.xlabel('CP1 concentration [M]')
# ax.zlabel('Normalized intensity')
ax.set_xlabel('CP1 concentration (M)', fontsize=16, rotation=0)
ax.set_ylabel('Time (s)', fontsize=15)
ax.set_zlabel('Norm. intensity', fontsize=15, rotation=0)
ax.set_zlim(0, 1)

# ax.ticklabel_format(axis='both', style='sci')
ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))


# plt.xscale('symlog', subsx=[1, 2, 3, 4, 5, 6, 7, 8, 9], linscalex=0.3, linthreshx=1e-5)
# plt.xscale('log')
# yaxis = plt.gca().yaxis
# yaxis.set_minor_locator(MinorSymLogLocator(100))

# plt.savefig('fig.pdf', format='pdf')
# plt.savefig('figure.png', dpi=1000, format='png', transparent=True)

plt.show()





NameError: name 'np' is not defined

In [289]:
curve_z[::10]


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [162]:
A = 0
plt.plot(conc, general(conc, 1.5e10, k_ex=0, A0=A, t_max=4e-6, intensity=True, curves=False, norm_curves=False), label='var')
# plt.plot(conc, general(conc, 2e-5, A, True), label='upper')
# plt.plot(conc, general(conc, 2e10, A, True), label='lower')
plt.legend()
plt.show()